In [1]:
import pandas as pd
import time
from selenium import webdriver

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
from selenium.webdriver.common.by import By

# load a web page in WebDriver
driver.get('https://www.list.am/category/60?n=1&price1=10000&price2=&crc=&_a5=0&_a39=0&_a40=0&_a85=0&_a73=0&_a3_1=&_a3_2=&_a4=0&_a37=0&_a36=0&_a11_1=&_a11_2=&_a41=0&_a78=0&_a38=0&_a82=0&_a77=0')

In [2]:
all_apartments = []

In [3]:
def change_lang():
    # change to eng
    lang = driver.find_elements(By.XPATH, '//*[@id="lbar"]')
    lang[0].click()
    time.sleep(1)
    # select english flag
    en_lang = driver.find_elements(By.XPATH, '//*[@id="lmenu"]/a[2]')
    en_lang[0].click()

In [4]:
change_lang()

In [5]:
def get_apartment_data():
    address = driver.find_element(By.XPATH, '//*[@id="abar"]/div[1]/a')
    price = driver.find_element(By.XPATH, '//*[@id="abar"]/div[2]/span[1]')
    building_details = driver.find_elements(By.XPATH, '//*[@id="pcontent"]/div/div[3]/div')[:4]
    apartment_details = driver.find_elements(By.XPATH, '//*[@id="pcontent"]/div/div[5]/div')[:8]

    apartment_data = {
        'address': address.text,
        'price': price.text
    }

    details = building_details + apartment_details
    for detail in details:
        key, value = detail.find_elements(By.CSS_SELECTOR, 'div')
        value_text = value.text
        if key.text == 'Floor Area':
            value_text = value_text.split(' ')[0]
        apartment_data[key.text] = value_text
    return apartment_data

In [6]:
def add_each_apartment(results):
    for result in results:
        province = result.find_element(By.CSS_SELECTOR, 'div.at')
        province_name = province.text
        province_name = province_name.split(',')[0]
        app_data = {
            'province': province_name
        }

        # open the page of apartment,  and get data about this apartment
        result.click()
        window_before = driver.window_handles[0]
        window_after = driver.window_handles[1]
        # go to the opened page(page of apartment)
        driver.switch_to.window(window_after)
        # get data about this apartment and add to dictionary
        app_data.update(get_apartment_data())
        # add data of one apartment to list of all apartments
        all_apartments.append(app_data)
        # close page of apartment and go to the main page
        driver.close()
        driver.switch_to.window(window_before)

In [9]:
def get_one_page_data():
    top_results = driver.find_elements(By.XPATH, '//*[@id="tp"]/div[2]/div/a')
    regular_results = driver.find_elements(By.XPATH, '//*[@id="contentr"]/div[4]/div[1]/a')
    results = top_results + regular_results
    add_each_apartment(results)

In [10]:
poit = 0

# get data from every page of pagination in every cycle
while poit < 11:
    get_one_page_data()

    # go to next pagination
    next_button = driver.find_elements(By.XPATH, '//*[@id="contentr"]/div[4]/div[2]/a')[-1]
    next_button.click()
    poit += 1

[<selenium.webdriver.remote.webelement.WebElement (session="de3da086b6f67b55c601fd646d39c457", element="7DA56B5110C325BA0B2E05B8391B29F5_element_434")>, <selenium.webdriver.remote.webelement.WebElement (session="de3da086b6f67b55c601fd646d39c457", element="7DA56B5110C325BA0B2E05B8391B29F5_element_435")>]


In [532]:
len(all_apartments)

1155

In [534]:
df1 = pd.DataFrame(all_apartments)
df1

,province,address,price,Construction Type,New Construction,Elevator,Floors in the Building,Floor Area,Number of Rooms,Number of Bathrooms,Ceiling Height,Floor,Balcony,Furniture,Renovation
0,Kentron,"Kristapor street 20, Yerevan","$158,000",Stone,No,Not available,5,50,3,1,from 3 m,4,Closed balcony,Available,Designer Renovation
1,Kentron,"Աբովյան փողոց, Երևան","$260,000",Stone,No,Not available,5,103,3,1,from 2.75 m,2,Closed balcony,Available,Cosmetic Renovation
2,Kentron,"Улица Сарьяна 38, Yerevan","$205,000",Monolith,No,Available,16,100,3,1,from 2.7 m,13,Multiple balconies,Partial Furniture,Partial Renovation
3,Kanaker-Zeytun,"Dro street, Yerevan","$119,000",Panels,No,Available,13,75,4,1,from 2.7 m,9,Not available,Available,Major Renovation
4,Erebuni,"Rostovyan street, Yerevan","$45,000",Stone,No,Not available,5,29,1,1,from 2.7 m,1,Not available,Not available,Old Renovation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,Kentron,"Agatangeghos Street, Yerevan","$193,800",Panels,No,Available,11,83,3,1,from 2.8 m,5,Open balcony,Available,Major Renovation
1151,Nor Nork,"Nansen street lane, Yerevan","$123,000",Panels,No,Not available,5,78,3,1,from 2.6 m,5,Closed balcony,Available,Cosmetic Renovation
1152,Nor Nork,"Minsk Street, Yerevan","$120,000",Panels,No,Available,9,93,4,2,from 2.75 m,7,Open balcony,Available,Partial Renovation
1153,Kentron,"Amiryan Street, Yerevan","$550,000",Stone,No,Not available,4,172,6,2,from 3.2 m,4,Open balcony,Available,Old Renovation


In [535]:
df1.to_csv('apartments1.csv', index=False)

In [542]:
driver.quit()